In [2]:
library(tidyverse)
library(repr)
library(rvest)
library(stringr)
library(janitor)
library(dplyr) 
library(ggplot2)
options(repr.matrix.max.rows = 6)
install.packages("IRdisplay")
install.packages("rmarkdown") 
library(IRdisplay)
library(rmarkdown)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘rvest’


The following object is masked from ‘package:readr’:

    guess_encoding



Attaching package: ‘janitor’


The following objects are masked from ‘package:stats’:

    chisq.test, fisher.test


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



# Loading in Datasets

In [3]:
player_link <- "https://raw.githubusercontent.com/Shervin2006/DSCI100-Project/refs/heads/main/players.csv"
session_link <- "https://raw.githubusercontent.com/Shervin2006/DSCI100-Project/refs/heads/main/sessions.csv"

# Read the CSV files into data frames
players <- read_csv(player_link)
sessions <- read_csv(session_link)

head(sessions)
head(players)

Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1535 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): hashedEmail, start_time, end_time
dbl (2): original_start_time, original_end_time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


hashedEmail                                                     
1 bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf
2 36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686
3 f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3398304c7ae42581fdc
4 bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf
5 36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686
6 bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf
  start_time       end_time         original_start_time original_end_time
1 30/06/2024 18:12 30/06/2024 18:24 1.71977e+12         1.71977e+12      
2 17/06/2024 23:33 17/06/2024 23:46 1.71867e+12         1.71867e+12      
3 25/07/2024 17:34 25/07/2024 17:57 1.72193e+12         1.72193e+12      
4 25/07/2024 03:22 25/07/2024 03:58 1.72188e+12         1.72188e+12      
5 25/05/2024 16:01 25/05/2024 16:12 1.71665e+12         1.71665e+12      
6 23/06/2024 15:08 23/06/2024 17:10 1.71916e+12         1.71916e+12

experience subscribe
1 Pro         TRUE    
2 Veteran     TRUE    
3 Veteran    FALSE    
4 Amateur     TRUE    
5 Regular     TRUE    
6 Amateur     TRUE    
  hashedEmail                                                      played_hours
1 f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d 30.3        
2 f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9  3.8        
3 b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28  0.0        
4 23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4fa7a5a659ff443a0eb5  0.7        
5 7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb0af4d48fcce2420f3e  0.1        
6 f58aad5996a435f16b0284a3b267f973f9af99e7a89bee0430055a44fa92f977  0.0        
  name      gender Age
1 Morgan    Male    9 
2 Christian Male   17 
3 Blake     Male   17 
4 Flora     Female 21 
5 Kylie     Male   21 
6 Adrian    Female 17

# Wrangling and Summary Values

In [4]:
player_data <- players |>
  filter(!is.na(Age) & !is.na(subscribe))

# Mean hours played relative to Experience
wrangled_player_data_hrs <- player_data |>
  group_by(experience) |>
  summarise(average_playtime_hrs = mean(played_hours, na.rm = TRUE)) |>
  arrange(desc(average_playtime_hrs)) |>
  slice_head(n = 6)
print(wrangled_player_data_hrs)

# Mean hours played relative to Gender
wrangled_player_data_gender <- player_data |>
  group_by(gender) |>
  summarise(average_playtime_hrs = mean(played_hours, na.rm = TRUE)) |>
  arrange(desc(average_playtime_hrs)) |>
  slice_head(n = 6)
print(wrangled_player_data_gender)

# Count the number of people per Gender
number_of_people_per_gender <- player_data |>
  group_by(gender) |>
  summarise(total_players = n())
print(number_of_people_per_gender)

# Mean hours played relative to Age
wrangled_player_data_age <- player_data |>
  group_by(Age) |>
  summarise(average_playtime_hrs = mean(played_hours, na.rm = TRUE)) |>
  arrange(desc(average_playtime_hrs)) |>
  slice_head(n = 6)
print(wrangled_player_data_age)

# Mean for each numeric variable: played_hours, Age, and subscription rate (using subscribe as logical)
mean_values <- player_data |>
  summarise(
    Mean_Played_Hours = mean(played_hours, na.rm = TRUE),
    Mean_Age = mean(Age, na.rm = TRUE),
    Mean_Subscription = mean(subscribe, na.rm = TRUE)  # TRUE is treated as 1, FALSE as 0
  )
print(mean_values)

# A tibble: 5 × 2
  experience average_playtime_hrs
  <chr>                     <dbl>
1 Regular                  18.7  
2 Amateur                   6.02 
3 Pro                       2.78 
4 Beginner                  1.25 
5 Veteran                   0.648
# A tibble: 6 × 2
  gender            average_playtime_hrs
  <chr>                            <dbl>
1 Non-binary                      14.9  
2 Female                          10.6  
3 Agender                          6.25 
4 Male                             4.13 
5 Prefer not to say                0.373
6 Two-Spirited                     0.08 
# A tibble: 6 × 2
  gender            total_players
  <chr>                     <int>
1 Agender                       2
2 Female                       37
3 Male                        124
4 Non-binary                   15
5 Prefer not to say            11
6 Two-Spirited                  5
# A tibble: 6 × 2
    Age average_playtime_hrs
  <dbl>                <dbl>
1    16                 50.0
2  